## Import libraries

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob

# global variables
OUTPUT_FOLDER = "./output_images/"
TEST_FOLDER = "./test_images/"


## Explore dataset

In [ ]:
images = glob.glob('*.jpeg')
cars = []
notcars = []

# Divide up into cars and notcars
images = glob.glob('./non-vehicles/*/*.png')
cars = []
notcars = []
for image in images:
        notcars.append(image)
images = glob.glob('./vehicles/*/*.png')
for image in images:
        cars.append(image)

# Helper function to plot graphs and figures
def plot_figures(figures, figSize, r = 1, c=1, fig_title=None, labels=None):
    fig, axs = plt.subplots(ncols=c, nrows=r, figsize=figSize)
    axs = axs.ravel()
    for index, title in zip(range(len(figures)), figures):
        axs[index].imshow(figures[title], plt.gray())
        if(labels != None):
            axs[index].set_title(labels[index])
        axs[index].set_axis_off()
    if labels is not None: plt.tight_layout()
    if fig_title is not None:
        fig.suptitle(fig_title, fontsize=16)

# Define a function to return some characteristics of the dataset 
def data_look(car_list, notcar_list):
    data_dict = {}
    # Define a key in data_dict "n_cars" and store the number of car images
    data_dict["n_cars"] = len(car_list)
    # Define a key "n_notcars" and store the number of notcar images
    data_dict["n_notcars"] = len(notcar_list)
    # Read in a test image, either car or notcar
    test_img = mpimg.imread(car_list[0])
    # Define a key "image_shape" and store the test image shape 3-tuple
    data_dict["image_shape"] = test_img.shape
    # Define a key "data_type" and store the data type of the test image.
    data_dict["data_type"] = test_img.dtype
    # Return data_dict
    return data_dict
    
data_info = data_look(cars, notcars)

print('There are ', 
      data_info["n_cars"], ' cars and', 
      data_info["n_notcars"], ' non-cars')
print('of size: ',data_info["image_shape"], ' and data type:', 
      data_info["data_type"])
# Just for fun choose random car / not-car indices and plot example images   
car_ind = np.random.randint(0, len(cars))
notcar_ind = np.random.randint(0, len(notcars))
    
# Read in car / not-car images
car_image = mpimg.imread(cars[car_ind])
notcar_image = mpimg.imread(notcars[notcar_ind])

# Display 48 randomly chosen car images from the dataset
labels = []
car_images = {}
for i in range(0, 48):
    idx = np.random.randint(0, len(cars))
    car_images[i] = mpimg.imread(cars[idx])
    labels.append(cars[idx])
plot_figures(car_images, (6,8), 8, 6, "Example Car Images", None)
plt.savefig(OUTPUT_FOLDER + "car_visualization.png")

# Display 48 randomly chosen noncar images from the dataset
labels = []
notcar_images = {}
for i in range(0, 48):
    idx = np.random.randint(0, len(notcars))
    notcar_images[i] = mpimg.imread(notcars[idx])
    labels.append(notcars[idx])
plot_figures(notcar_images, (6,8), 8, 6, "Example Not-Car Images", None)
plt.savefig(OUTPUT_FOLDER + "notcar_visualization.png")


## Preprocessing

I noticed that the SVM does poorly on patches of images that are dimly lit (in shadows etc.), hence we use histogram equalization on the images before feature extraction. Here is a visualization of the results. We can see that histogram equalization is able to bring out the features in an image - the colors, edges, gradient and shape. Note that although traditionally histogram equalization is only to be used on grayscale image, here we treat each channel of a colorspace as a separate historgram to be equalized.


In [ ]:
# input image must be a single channel image
def equalize_hist(img):
    if img.dtype == 'float32' or img.dtype == 'float64':
        img = (img * 255).astype(np.uint8)
    img = np.reshape(img, (img.shape[0], img.shape[1]))
    return cv2.equalizeHist(img)

## Extract features

In [ ]:
# Define a function to extract features from a list of images
# This will extract color_histograms and hog features
def extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
                     hist_bins=32, hist_range=(0, 256), orient=9, 
                     pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        feature_image = convert_color(image, cspace)

        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        
        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                feature_image_equalized = equalize_hist(feature_image[:,:,channel])
                hog_features.append(get_hog_features(feature_image_equalized, 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)
        else:
            feature_image_equalized = equalize_hist(feature_image[:,:,hog_channel])
            hog_features = get_hog_features(feature_image_equalized, orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features, hog_features)))
#         features.append(hog_features)

    # Return list of feature vectors
    return features

def bin_spatial(img, size=(32, 32)):
    return cv2.resize(img, size).ravel()

# Function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to return HOG features and visualization
from skimage.feature import hog
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features
    
def convert_color(image, cspace='RGB'):
    if cspace != 'RGB':
        if cspace == 'HSV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        elif cspace == 'LUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
        elif cspace == 'HLS':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        elif cspace == 'YUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        else:
            feature_image = np.copy(image)
    else:
        feature_image = np.copy(image)
    return feature_image

## Visualize HOG
Here we try out different color spaces and compare their visualizations to see which color spaces and which channels show the most contrast and therefore, are the best to use for feature extraction. It was noticed later on that a lot of false positives were showing up and those are patches of road / guardrail that were in the shadows. Hence, we can use this function to chery pick non-car images that the SVM is getting wrong and figure out why.

Based on the plots below, it seems that S and V channel in the HSV colorspace provides the greatest contrast between car and non-car images.

In [ ]:
# Constants
spatial_size = (32,32)
hist_bins = 32
colorspace = 'HSV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"

images = glob.glob('./non-vehicles/*/image3689.png')
cars = []
notcars = []
for image in images[0:1]:
        notcars.append(image)
images = glob.glob('./vehicles/*/*.png')
for image in images[0:1]:
        cars.append(image)

cspaces = ["RGB", "HSV", "LUV", "YCrCb", "HLS", "YUV"]
for car_filename, notcar_filename in zip(cars, notcars):
    image = mpimg.imread(car_filename)
    notcar_image = mpimg.imread(notcar_filename)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = equalize_hist(gray)
    notcar_gray = cv2.cvtColor(notcar_image, cv2.COLOR_RGB2GRAY)
    notcar_gray = equalize_hist(notcar_gray)
    fig, axes = plt.subplots(12,4, figsize=(8,24))
    for row, cspace in enumerate(cspaces):
        feature_image = convert_color(image, cspace)
        notcar_feature_image = convert_color(notcar_image, cspace)
        axes[2*row,0].imshow(gray, cmap='gray')
        axes[2*row,0].set_title("Original car")
        axes[2*row+1,0].imshow(notcar_gray, cmap='gray')
        axes[2*row+1,0].set_title("Original not car")
        for channel in range(feature_image.shape[2]):
            feature_equalized = equalize_hist(feature_image[:,:,channel])
            features, hog_image = get_hog_features(feature_equalized, 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=True, feature_vec=True)
            axes[2*row,channel+1].imshow(hog_image, cmap='gray')
            axes[2*row,channel+1].set_title(cspace + "_car_hog_%d" % channel)
        for channel in range(feature_image.shape[2]):
            notcar_feature_equalized = equalize_hist(notcar_feature_image[:,:,channel])
            features, hog_image = get_hog_features(notcar_feature_equalized, 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=True, feature_vec=True)
            axes[2*row+1,channel+1].imshow(hog_image, cmap='gray')
            axes[2*row+1,channel+1].set_title(cspace + "_noncar_hog_%d" % channel)
        fig.tight_layout()
    fig.savefig(OUTPUT_FOLDER + "hog_colorspace_comparision.png")

## Train the classifer using SVM

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import time

# Divide up into cars and notcars
images = glob.glob('./non-vehicles/*/*.png')
cars = []
notcars = []
for image in images:
        notcars.append(image)
images = glob.glob('./vehicles/*/*.png')
for image in images:
        cars.append(image)

# Reduce the sample size because HOG features are slow to compute
sample_size = 8000
cars = cars[0:sample_size]
notcars = notcars[0:sample_size]

### TODO: Tweak these parameters and see how the results change.
spatial_size = (32,32)
hist_bins = 32
colorspace = 'HSV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"

t=time.time()
car_features = extract_features(cars, cspace=colorspace, spatial_size=(32,32),
                        hist_bins=32, hist_range=(0,256), orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_features = extract_features(notcars, cspace=colorspace, spatial_size=(32,32),
                        hist_bins=32, hist_range=(0,256), orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)

print("Training with %d features on %d positive-datapoints and %d negative datapoints" % (car_features[0].shape[0], sample_size, sample_size))

t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

In [ ]:
from sklearn.svm import SVC
from sklearn import grid_search

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_state)
    
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)

# Apply the scaler to X
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

t=time.time()

# Grid search the best model
parameters = {'kernel':('linear', 'rbf'), 'C':[0.1, 1, 10, 100]}
svr = SVC()
clf = grid_search.GridSearchCV(svr, parameters)
clf.fit(X_train, y_train)
svc = clf
# Use a linear SVC 
# svc = LinearSVC()

# Check the training time for the SVC
# svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to find best parameters and train SVC...')
# Check the score of the SVC
print('Best Train Accuracy of SVC = ', round(svc.score(X_train, y_train), 4))
print('Best Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
best_params = svc.best_params_
print('Best parameters for SVC: ' + str(best_params))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

## Sliding windows

In [ ]:
# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, cspace, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    bboxes = []
    
    img_tosearch = img[ystart:ystop,:,:]
    ctrans_tosearch = convert_color(img_tosearch, cspace)
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    
    # Compute individual channel HOG features for the entire image
#     hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
#     hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
#     hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
#     print(ch3.shape)
#     print(hog3.shape)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell
            
            # Histogram equalization
            window1 = equalize_hist(ch1[ytop:ytop+window, xleft:xleft+window])
            window2 = equalize_hist(ch2[ytop:ytop+window, xleft:xleft+window])
            window3 = equalize_hist(ch3[ytop:ytop+window, xleft:xleft+window])
    
            # Extract HOG for this patch
#             hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
#             hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
#             hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
#             print(window1.shape, window2.shape, window3.shape)
            hog_feat1 = get_hog_features(window1, orient, pix_per_cell, cell_per_block, feature_vec=True)
            hog_feat2 = get_hog_features(window2, orient, pix_per_cell, cell_per_block, feature_vec=True)
            hog_feat3 = get_hog_features(window3, orient, pix_per_cell, cell_per_block, feature_vec=True)
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

#             print(hog_feat1.shape, hog_feat2.shape, hog_feat3.shape)
            

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
#             test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))
            
            test_features = X_scaler.transform(hog_features.reshape(1, -1))
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                # print(svc.decision_function(test_features))
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                top_left = (xbox_left, ytop_draw+ystart)
                bottom_right = (xbox_left+win_draw,ytop_draw+win_draw+ystart)
                bboxes.append((top_left, bottom_right))
                cv2.rectangle(draw_img, top_left, bottom_right,(0,0,255),6) 
                
    return draw_img, bboxes


In [ ]:
import os

ystart = 400
ystop = 656
scale = 1.5

start = time.time()
images = glob.glob(TEST_FOLDER + "*.jpg")
for filename in images:
    img = mpimg.imread(filename)
    out_img, bboxes = find_cars(img, ystart, ystop, scale, colorspace, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    plt.figure()
    plt.imshow(out_img)
    plt.imsave(OUTPUT_FOLDER + "sliding_windows_" + filename.split(os.path.sep)[-1], out_img)
end = time.time() - start
print("%.2fs taken for %d images" % (end, len(images)))

## Heatmap
1. Draw a heatmap based on list of detections
2. Apply a threshold to remove false positives
3. Draw bounding boxes on the final detected cars

In [ ]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap

def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

from scipy.ndimage.measurements import label

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

images = glob.glob(TEST_FOLDER + "*.jpg")
for filename in images:
    img = mpimg.imread(filename)
    out_img, bboxes = find_cars(img, ystart, ystop, scale, colorspace, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    # Add heat to each box in box list
    heat = add_heat(heat, bboxes)
    
    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,3)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)
    plt.imsave(OUTPUT_FOLDER + "heatmap_" + filename.split(os.path.sep)[-1], heatmap)
    
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    final_img = draw_labeled_bboxes(np.copy(img), labels)
    plt.figure()
    plt.imshow(final_img)
    plt.imsave(OUTPUT_FOLDER + "final_detected_cars_" + filename.split(os.path.sep)[-1], final_img)